In [1]:
import glob

import numpy as np
import pandas as pd

import xarray as xr

import plotly.express as px
import plotly.graph_objects as go
import matplotlib.pyplot as plt

In [2]:
df = pd.read_parquet('../data/projections_gares/')
df['tasmaxAdjust'] = df['tasmaxAdjust'] - 273.15
df['year'] = df['time'].dt.year
df = df.sort_values(by=['time', 'lat', 'lon'])
df

,time,tasmaxAdjust,lon,lat,gare,year
994,2015-06-01 12:00:00,29.417877,1.9421,42.433,Bourg-Madame,2015
963,2015-06-01 12:00:00,24.430481,3.1584,42.436,Cerbère,2015
1057,2015-06-01 12:00:00,27.631195,1.9106,42.454,Latour-de-Carol - Enveitg,2015
1059,2015-06-01 12:00:00,25.184479,2.0322,42.457,Saillagouse,2015
1128,2015-06-01 12:00:00,24.553925,2.0311,42.480,Font-Romeu-Odeillo-Via,2015
...,...,...,...,...,...,...
8027214,2099-08-31 12:00:00,22.738495,2.4166,50.960,Bergues,2099
8027226,2099-08-31 12:00:00,22.511292,2.1311,50.977,Gravelines,2099
8027248,2099-08-31 12:00:00,22.586365,2.3078,51.003,Grande-Synthe,2099
8027261,2099-08-31 12:00:00,22.658966,2.3778,51.027,Coudekerque-Branche,2099


In [3]:
df['lag_1'] = df['tasmaxAdjust'].shift(1)
df['lag_2'] = df['tasmaxAdjust'].shift(2)

mask_cond = (
    (df['tasmaxAdjust'] > 40) &
    (df['lag_1'] > 35) &
    (df['lag_2'] > 35)
)
df_cond_day = df[mask_cond]
df_cond_day.to_csv("../data/projection_tasmax_cond.csv", index=False)

df_cond_day = df_cond_day.drop(['lag_1', 'lag_2', 'year'], axis=1)

df_cond_day

,time,tasmaxAdjust,lon,lat,gare
2543620,2017-06-30 12:00:00,40.267120,4.8625,43.997,Sorgues - Châteauneuf-du-Pape
2543622,2017-06-30 12:00:00,40.403168,4.9249,43.997,Entraigues-sur-la-Sorgue
2544199,2017-06-30 12:00:00,40.510590,4.8938,44.042,Bédarrides
2544202,2017-06-30 12:00:00,40.940979,4.9875,44.042,Monteux
2544204,2017-06-30 12:00:00,40.945343,5.0500,44.042,Carpentras
...,...,...,...,...,...
6663017,2099-08-16 12:00:00,40.459778,4.7679,45.593,Givors Canal
6663018,2099-08-16 12:00:00,40.128906,4.8000,45.593,Chasse-sur-Rhône
6664596,2099-08-16 12:00:00,40.311615,0.1658,45.656,Angoulême
7423190,2099-08-25 12:00:00,40.520844,6.4704,43.445,Les Arcs - Draguignan


In [4]:
path_cond_min = "../data/projection_tasmin_cond.csv"
df_cond_night = pd.read_csv(path_cond_min)
df_cond_night['time'] = pd.to_datetime(df_cond_night['time'], yearfirst=True)
df_cond_night.drop(['Unnamed: 0', 'index', 'two_consecutive_nights', 'lat_y', 'lon_y'], axis=1, inplace=True)
df_cond_night.rename(columns={'lon_x': 'lon', 'lat_x': 'lat'}, inplace=True)
df_cond_night

,time,tasminAdjust,lon,lat,gare
0,2015-06-25 12:00:00,20.525177,3.1584,42.436,Cerbère
1,2015-07-09 12:00:00,22.949799,3.1584,42.436,Cerbère
2,2015-07-10 12:00:00,21.773010,3.1584,42.436,Cerbère
3,2015-07-11 12:00:00,20.451600,3.1584,42.436,Cerbère
4,2015-07-12 12:00:00,20.744904,3.1584,42.436,Cerbère
...,...,...,...,...,...
1292508,2099-08-14 12:00:00,21.571198,2.3778,51.027,Dunkerque
1292509,2099-08-15 12:00:00,20.573029,2.3778,51.027,Coudekerque-Branche
1292510,2099-08-15 12:00:00,20.573029,2.3778,51.027,Dunkerque
1292511,2099-08-16 12:00:00,20.912811,2.3778,51.027,Coudekerque-Branche


In [5]:
df_cond = pd.merge(
    left=df_cond_day, right=df_cond_night, how='left', on=['time', 'lon', 'lat', 'gare']
)
df_cond = df_cond.dropna()
df_cond['year'] = df_cond['time'].dt.year
df_cond.to_csv("../data/projection_cond_globale.csv")

df_cond

,time,tasmaxAdjust,lon,lat,gare,tasminAdjust,year
0,2017-06-30 12:00:00,40.267120,4.8625,43.997,Sorgues - Châteauneuf-du-Pape,22.751709,2017
1,2017-06-30 12:00:00,40.403168,4.9249,43.997,Entraigues-sur-la-Sorgue,21.857330,2017
2,2017-06-30 12:00:00,40.510590,4.8938,44.042,Bédarrides,22.799774,2017
3,2017-06-30 12:00:00,40.940979,4.9875,44.042,Monteux,22.024414,2017
4,2017-06-30 12:00:00,40.945343,5.0500,44.042,Carpentras,20.589417,2017
...,...,...,...,...,...,...,...
55000,2099-08-16 12:00:00,40.459778,4.7679,45.593,Givors,21.810760,2099
55001,2099-08-16 12:00:00,40.459778,4.7679,45.593,Givors Canal,21.810760,2099
55002,2099-08-16 12:00:00,40.128906,4.8000,45.593,Chasse-sur-Rhône,22.124450,2099
55003,2099-08-16 12:00:00,40.311615,0.1658,45.656,Angoulême,21.228088,2099


In [6]:
trac_dates_1 = range(2042, 2062)
trac_dates_2 = range(2068, 2088)

df_trac_1 = df_cond[df_cond['year'].isin(trac_dates_1)]
df_trac_2 = df_cond[df_cond['year'].isin(trac_dates_2)]

df_trac_1 = df_trac_1.groupby(['year', 'gare', 'lat', 'lon'])['tasmaxAdjust'].count().reset_index()
df_trac_2 = df_trac_2.groupby(['year', 'gare', 'lat', 'lon'])['tasmaxAdjust'].count().reset_index()
df_trac_1.rename(columns={'tasmaxAdjust': "Nombre d'occurence"}, inplace=True)
df_trac_2.rename(columns={'tasmaxAdjust': "Nombre d'occurence"}, inplace=True)

print("shape tracc 1:", df_trac_1.shape)
print("shape tracc 2:", df_trac_2.shape)

df_trac_1

shape tracc 1: (2419, 5)
shape tracc 2: (6655, 5)


,year,gare,lat,lon,Nombre d'occurence
0,2042,Agen,44.211,0.6289,2
1,2042,Agonac,45.295,0.7398,2
2,2042,Aiguillon,44.290,0.3402,2
3,2042,Aixe-sur-Vienne,45.804,1.1545,2
4,2042,Alaï,45.750,4.7678,2
...,...,...,...,...,...
2414,2060,Strasbourg Roethig,48.570,7.6861,1
2415,2060,Toul,48.690,5.8888,1
2416,2060,Vendenheim,48.659,7.7250,1
2417,2060,Waligator Parc,49.226,6.1751,1


In [12]:
fig = px.scatter_map(data_frame=df_trac_1, lat='lat', lon='lon', animation_group="gare", animation_frame="year", 
                     color="Nombre d'occurence", size="Nombre d'occurence", range_color=[0, 8],
                     color_continuous_scale=["white", "yellow", "orange", "purple"],
                     zoom=4.5, height=700, width=900,
                     title="Nombre de jours où la condition est atteinte par année, TRACC 2°C"
)
fig.show()

In [13]:
fig = px.scatter_map(data_frame=df_trac_2, lat='lat', lon='lon', animation_group="gare", animation_frame="year", 
                     color="Nombre d'occurence", size="Nombre d'occurence", range_color=[0, 8],
                     color_continuous_scale=["white", "yellow", "orange", "purple"],
                     zoom=4.5, height=700, width=900,
                     title="Nombre de jours où la condition est atteinte par année, TRACC 2,7°C"
)
fig.show()

In [ ]:
df_to_plot = df_cond.groupby(['year']).agg({'tasmaxAdjust': 'count', 'gare': 'unique'}).reset_index()
df_to_plot.rename(columns={'tasmaxAdjust': 'Nombre d\'occurence', 'gare': 'Nombre de gares'}, inplace=True)
df_to_plot['Nombre de gares'] = df_to_plot['Nombre de gares'].apply(lambda x: len(x))
df_to_plot['mean'] = df_to_plot["Nombre d\'occurence"].rolling(10).mean()
df_to_plot

,year,Nombre d'occurence,Nombre de gares,mean
0,2017,54,52,NaN
1,2020,2,2,NaN
2,2021,19,11,NaN
3,2022,8,8,NaN
4,2023,797,462,NaN
...,...,...,...,...
64,2095,89,85,835.5
65,2096,104,66,845.4
66,2097,1099,700,942.3
67,2098,704,321,957.6


In [ ]:
fig = px.line(data_frame=df_to_plot.dropna(), x='year', y=['Nombre d\'occurence', 'mean'])
fig.add_vrect(
    x0=2042, x1=2061,
    fillcolor="orange", opacity=0.15,
    layer="below", line_width=0,
    annotation_text="Période 2042-2061 : TRACC 2°C", annotation_position="top left"
)
fig.add_vrect(
    x0=2068, x1=2087,
    fillcolor="purple", opacity=0.15,
    layer="below", line_width=0,
    annotation_text="Période 2068-2087 : TRACC 2.7°C", annotation_position="top left"
)
title = "Nombre d'occurences où la condition est atteinte"
subtitle = "Condition : 2 nuits > 20°C, 3 jours > 35°C dont le 3ème > 40°C"
fig.update_layout(
    title={
        'text': f"{title}<br><span style='font-size:12px;'>{subtitle}</span>",
    }
)
fig.update_yaxes(title="Nombre d'occurence")
fig.update_xaxes(title="Année")
fig.show()

In [ ]:
df_to_plot = df_cond.groupby(['time']).agg({'tasmaxAdjust': 'count', 'gare': 'unique'}).reset_index()
df_to_plot.rename(columns={'tasmaxAdjust': 'Nombre d\'occurence', 'gare': 'Nombre de gares'}, inplace=True)
df_to_plot['Nombre de gares'] = df_to_plot['Nombre de gares'].apply(lambda x: len(x))
df_to_plot['mean'] = df_to_plot["Nombre d\'occurence"].rolling(10).mean()
df_to_plot

,time,Nombre d'occurence,Nombre de gares,mean
0,2017-06-30 12:00:00,8,8,NaN
1,2017-07-13 12:00:00,9,9,NaN
2,2017-07-15 12:00:00,37,37,NaN
3,2020-06-20 12:00:00,2,2,NaN
4,2021-08-10 12:00:00,1,1,NaN
...,...,...,...,...
686,2099-08-13 12:00:00,1,1,336.0
687,2099-08-14 12:00:00,9,9,335.9
688,2099-08-15 12:00:00,205,205,355.8
689,2099-08-16 12:00:00,29,29,337.3


In [ ]:
fig = px.line(data_frame=df_to_plot.dropna()[df_to_plot['time'] > pd.to_datetime("2030-01-01", yearfirst=True)],
              x='time', y=['Nombre d\'occurence'])#, 'mean'])
fig.add_vrect(
    x0=pd.to_datetime("2042-01-01", yearfirst=True), x1=pd.to_datetime("2061-01-01", yearfirst=True),
    fillcolor="orange", opacity=0.15,
    layer="below", line_width=0,
    annotation_text="Période 2042-2061 : TRACC 2°C", annotation_position="top left"
)
fig.add_vrect(
    x0=pd.to_datetime("2068-01-01", yearfirst=True), x1=pd.to_datetime("2087-01-01", yearfirst=True),
    fillcolor="purple", opacity=0.15,
    layer="below", line_width=0,
    annotation_text="Période 2068-2087 : TRACC 2.7°C", annotation_position="top left"
)
title = "Nombre de gares où la condition est atteinte"
subtitle = "Condition : 2 nuits > 20°C, 3 jours > 35°C dont le 3ème > 40°C"
fig.update_layout(
    title={
        'text': f"{title}<br><span style='font-size:12px;'>{subtitle}</span>",
    }
)
fig.update_yaxes(title="Nombre d'occurence")
fig.update_xaxes(title="Année")
fig.show()

/var/folders/q4/hb535nv14kx3r_6j_jr93thh0000gq/T/ipykernel_78501/1761291231.py:1: UserWarning:

Boolean Series key will be reindexed to match DataFrame index.



In [26]:
date = "2082-07-01 12:00:00"
date = "2099-08-10 12:00:00"
df_to_plot = df_cond[df_cond['time'] == date]
df_to_plot = df_to_plot.sort_values('tasmaxAdjust')
df_to_plot

,time,tasmaxAdjust,lon,lat,gare,tasminAdjust,year
52811,2099-08-10 12:00:00,40.001068,6.8034,47.689,Bas Évette,22.459595,2099
53166,2099-08-10 12:00:00,40.002869,2.0390,48.775,Saint-Quentin en Yvelines - Montigny-le-Breton...,22.198303,2099
52594,2099-08-10 12:00:00,40.008575,5.1959,47.054,Pagny-le-Château,22.794342,2099
53500,2099-08-10 12:00:00,40.009064,3.4523,49.496,Anizy - Pinon,22.005615,2099
52956,2099-08-10 12:00:00,40.014648,1.6262,48.271,Voves,23.064514,2099
...,...,...,...,...,...,...,...
52802,2099-08-10 12:00:00,44.482086,3.7322,47.656,Vermenton,22.585754,2099
52827,2099-08-10 12:00:00,44.543915,3.5968,47.745,Champs - Saint-Bris,22.745026,2099
52807,2099-08-10 12:00:00,44.749084,3.6650,47.678,Cravant - Bazarnes,22.827942,2099
52817,2099-08-10 12:00:00,44.851440,3.6312,47.700,Vincelles,22.607880,2099


In [27]:
fig = px.scatter_map(data_frame=df_to_plot, lat='lat', lon='lon',
                     color="tasmaxAdjust", size="tasmaxAdjust", range_color=[40, 46],
                     color_continuous_scale=["white", "yellow", "orange", "purple"],
                     size_max=10,
                     zoom=4.5, height=700, width=900,
                     title="Nombre de jours où la condition est atteinte par année, TRACC 2,7°C"
)
fig.show()